In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from keras.models import load_model
path='drive/My Drive/deep_proj/' #for new data change the path
image_model_transfer=load_model(path+'image_model_transfer.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
from keras.models import model_from_json
# load json and create model
json_file = open(path+'decoder_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
decoder_model = model_from_json(loaded_model_json)
# load weights into new model
decoder_model.load_weights(path+"decoder_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
import pickle
# loading tokenizer
with open(path+'tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [0]:
import json

with open(path+'image_captioning_info.json') as json_file:
    image_captioning_info = json.load(json_file)

In [0]:
pip install pygame

     |████████████████████████████████| 11.4MB 3.5MB/s 


In [0]:
#load test data
import io, pygame
import zipfile
import cv2
import matplotlib.pyplot as plt
import numpy as np

image_number=39 #len(image_captioning_info)
data_set=np.zeros([image_number,224,224,3],dtype='uint8')
file_path=path+'captioning_images.zip'

labels=[]
archive = zipfile.ZipFile(file_path, 'r')
ind=0
for index in range(image_number):
    imgdata = archive.read(str(index+1)+'.jpg')  # image_captioning_info[index]['file_name']

    # create a pygame-compatible file-like object from the bytes
    bytes_io = io.BytesIO(imgdata)
    img = pygame.image.load(bytes_io)
    imgg = pygame.surfarray.array3d(img)
    final=imgg.swapaxes(0,1)
    resized = cv2.resize(final, (224,224), interpolation = cv2.INTER_AREA)
    data_set[index,:,:,:] = resized


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [0]:
import keras
data_set_preprocessed = keras.applications.xception.preprocess_input(data_set)

In [0]:
import matplotlib.pyplot as plt
import numpy as np
def generate_caption(im_idx, max_tokens=30):

    # Load the image.
    image = data_set_preprocessed[im_idx]
    
    # Expand the 3-dim numpy array to 4-dim
    # because the image-model expects a whole batch as input,
    # so we give it a batch with just one image.
    image_batch = np.expand_dims(image, axis=0)

    # Process the image with the pre-trained image-model
    # to get the transfer-values.
    transfer_values = image_model_transfer.predict(image_batch)

    # Pre-allocate the 2-dim array used as input to the decoder.
    # This holds just a single sequence of integer-tokens,
    # but the decoder-model expects a batch of sequences.
    shape = (1, max_tokens)
    decoder_input_data = np.zeros(shape=shape, dtype=np.int)

    # The first input-token is the special start-token 
    token_int = tokenizer.word_index['<start>']

    # Initialize an empty output-text.
    output_text = ''

    # Initialize the number of tokens we have processed.
    count_tokens = 0

    # While we haven't sampled the special end-token 
    # and we haven't processed the max number of tokens.
    while token_int != tokenizer.word_index['<end>'] and count_tokens < max_tokens:
        # Update the input-sequence to the decoder
        # with the last token that was sampled.
        # In the first iteration this will set the
        # first element to the start-token.
        decoder_input_data[0, count_tokens] = token_int

        # Wrap the input-data in a dict for clarity and safety,
        # so we are sure we input the data in the right order.
        x_data = \
        {
            'transfer_values_input': transfer_values,
            'decoder_input': decoder_input_data
        }

     
        
        # Input this data to the decoder and get the predicted output.
        decoder_output = decoder_model.predict(x_data)

        # Get the last predicted token as a one-hot encoded array.
        # Note that this is not limited by softmax, but we just
        # need the index of the largest element so it doesn't matter.
        token_onehot = decoder_output[0, count_tokens, :]

        # Convert to an integer-token.
        token_int = np.argmax(token_onehot)

        # Lookup the word corresponding to this integer-token.
        sampled_word = tokenizer.index_word[token_int]

        # Append the word to the output-text.
        output_text += " " + sampled_word

        # Increment the token-counter.
        count_tokens += 1

    # This is the sequence of tokens output by the decoder.
    output_tokens = decoder_input_data[0]

    # Plot the image.
    plt.imshow(image)
    plt.show()
    
    # Print the predicted caption.
    print("Predicted caption:")
    print(output_text)
    print()
    return output_text

In [0]:
generated_captions=[]
for i in range(data_set.shape[0]):
    caption=generate_caption(i)
    generated_captions.append(caption)

In [0]:
with open('/content/drive/My Drive/deep_proj/captions.txt', 'w') as output:
    output.write(str(generated_captions))